# Exercise 1: Query Expansion and Indexing

The following code is modified from Exercise 1. It is used to construct the vocabulary and vectorize the documents and query.

In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import numpy as np

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
#with open("bread.txt") as f:
with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(min(k,len(original_documents))):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to /Users/elmas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Question 1 - Relevance Feedback

In this exercise we will implement and test Rocchio's method for user relevance feedback.

Let the set of relevant documents to be $D_r $ and the set of non-relevant documents to be $D_{nr}$. Then the modified query  $\vec{q_m}$  according to the Rocchio method is:

\begin{equation}
\vec{q_m} = \alpha \vec{q_0} + \frac{\beta}{|D_r|} \sum_{\vec{d_j} \in D_r} \vec{d_j} - \frac{\gamma}{|D_{nr}|} \sum_{\vec{d_j} \in D_{nr}} \vec{d_j}
\end{equation}
In the Rocchio algorithm negative term weights are ignored. This means, for the negative term weights in $\vec{q_m}$, we set them to be 0.

First, complete the implementation of the Rocchio relevance feedback method, by adding the missing code for the function $expand\_query$.   

Then, compare the result obtained with the new query with the unmodified one.

In [ ]:
def expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, alpha, beta, gamma):
    
    num_rel = len(relevant_doc_vecs)
    num_non_rel = len(non_relevant_doc_vecs)
    
    # Compute the first term in the Rocchio equation
    norm_query_vector = ...
    
    # Compute the second term in the Rocchio equation
    norm_sum_relevant = ...
    
    # Compute the last term in the Rocchio equation
    norm_sum_non_relevant = ...
    
    # Sum all the terms
    modified_query_vector = ...
    
    # Ignore negative elements
    modified_query_vector = ...
    return modified_query_vector

In [ ]:
ans, result_doc_ids, query_vector = search_vec("computer science", 5)
for i in range(len(ans)):
    print(i,ans[i])

In [ ]:
# list of indices marked as relevant
# suppose first three documents were relevant and the rest were irrelevant.
relevant_indices = [0,1,2]
non_relevant_indices = [i for i in range(3, len(ans))]

relevant_doc_ids = [result_doc_ids[i] for i in relevant_indices]
non_relevant_doc_ids = [result_doc_ids[i] for i in non_relevant_indices]

relevant_doc_vecs = [document_vectors[i] for i in relevant_doc_ids]
non_relevant_doc_vecs = [document_vectors[i] for i in non_relevant_doc_ids]

expanded_query = expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, 1, 1, 1)

new_query = ' '.join([vocabulary[i] for i, val in enumerate(expanded_query) if val>0])

new_ans , not_important_now, idontcare_anymore = search_vec(new_query, 10)

print('Modified query: ', new_query)
new_ans

# Exercise 2: Link based ranking

### Preliminaries
If you want to normalize a vector to L1-norm or L2-norm, use:

In [ ]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

## Question 1 - Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

#### Hints:
We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [ ]:
# Implement your code here
def pagerank_eigen(L):
#   Construct transition probability matrix from L
    R = ...
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = ...
#     Take the eigen-vector with maximum eigven-value
    p = ...
    return (R,p)

In [ ]:
# Test with the question, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

# Question 2 - Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [ ]:
def pagerank_iterative(L):
    R = ...
    N = L.shape(0)
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = ...
        p = p + ...
        delta = ...
        i += 1

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

#### Test with the dataset


In [ ]:
# Construct link matrix from file
f = open("ca-GrQc.txt")
L = ...

In [ ]:
# Run PageRank
R, p = pagerank_iterative(L)
print("Ranking vector: p={0}".format(p[:,0]))

## Question 3 - Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

## Question 4 - Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes.

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [ ]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]

    x0, y0 = 1 / (N*N) * np.ones(N), 1 / (N*N) * np.ones(N) 

    xprev, yprev = x0, y0
    
    # For advanced exercise: define a convergence condition instead of k iterations
    for l in range(0,k):
        y = ...
        x = ...
        xprev = ...
        yprev = ...
        
    return xprev, yprev